In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pipelines.util.platform import start_spark
from pipelines.util.storage import read_csv,read_csvfolder,write_partby_parquet

In [ ]:
sps, logger, conf = start_spark(app_name='process_wtdata')

local


In [3]:
sch = T.StructType([
    T.StructField("timestamp", T.TimestampType(), True),
    T.StructField("turbine_id", T.IntegerType(), True),
    T.StructField("wind_speed", T.DoubleType(), True),
    T.StructField("wind_direction", T.IntegerType(), True),
    T.StructField("power_output", T.DoubleType(), True)
    ])

csvoptions = {
    "header": "true",
    "sep": ",",
    "schema": sch,
    "inferSchema": "false"
}

df_wtdata = read_csvfolder(sps, "{storage_landed}".format(storage_landed = sps.conf.get('storage.landed')), csvoptions)

In [4]:
df_wtdata = df_wtdata.withColumn("outputdate",F.to_date(F.col("timestamp")))

In [ ]:
df_wtdata.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- turbine_id: integer (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- wind_direction: integer (nullable = true)
 |-- power_output: double (nullable = true)
 |-- outputdate: date (nullable = true)



In [13]:
#df_wtdata.show(truncate=False)
#df_wtdata.count()
df_wtdata.agg(
    F.min(F.col("wind_speed")),F.max(F.col("wind_speed")),
    F.min(F.col("power_output")),F.max(F.col("power_output")),
    F.min(F.col("wind_direction")),F.max(F.col("wind_direction")),
).show()

# {
#     "wind_speed": "min",
#     "wind_speed": "max",
#     "power_output": "min",
#     "power_output": "max"
#     }

+---------------+---------------+-----------------+-----------------+-------------------+-------------------+
|min(wind_speed)|max(wind_speed)|min(power_output)|max(power_output)|min(wind_direction)|max(wind_direction)|
+---------------+---------------+-----------------+-----------------+-------------------+-------------------+
|            9.0|           15.0|              1.5|              4.5|                  0|                359|
+---------------+---------------+-----------------+-----------------+-------------------+-------------------+



In [6]:
write_partby_parquet(sps, df_wtdata, ["outputdate","turbine_id"], "{storage_landed}".format(storage_landed = sps.conf.get('storage.extracted')))